In [ ]:
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
##from sklearn.compose import ColumnTransformer, make_column_transformer
#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
#from sklearn import set_config; set_config(display='diagram')
from tensorflow.keras.preprocessing.text import Tokenizer
import string
import os
#import nltk
#nltk.download('stopwords')
#from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords 
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, Sequential
import tensorflow as tf

import tensorflow_hub as hub

import tensorflow_text as text

from tensorflow.keras.callbacks import EarlyStopping

#  Load the dataset already cleaned


In [ ]:
data = pd.read_csv("/Users/lucaspicot/code/Zen1400/fake_news/raw_data/cleaned_df.csv")

In [ ]:
## Delete the first column

data = data.drop(columns= 'Unnamed: 0')

In [ ]:
data = data.dropna()

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['text'],data['label'])

In [ ]:
X_train.head(4)

# Now lets import BERT model and get embeding vectors for few sample statements


In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

# Build Model



In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:

model.summary()

In [ ]:

len(X_train)

In [ ]:
es = EarlyStopping(patience = 5, verbose=2, monitor='val_loss', restore_best_weights = True)

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
] 

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_split = 0.20, callbacks = [es])

In [ ]:

model.evaluate(X_test, y_test)